# Introduction

Avant de commencer, parcourer le fichier README.rst

# Requête HTTP 

Un requête HTTP est une requête basé sur le protocole TCP, elle fait partie de la couche application de la couche OSI. Elle permet d'accéder aux données mise à disposition sur une adresse IP (ou url résolue par un DNS) et un port. 

Les deux ports les plus utilisé dans le web sont le 80 pour les sites en HTTP et le 443 pour les sites en HTTPS. HTTPS est une variable du protocole HTTP basé sur le protocole TLS.

Il existe de nombreux types de requêtes selon la convention `REST`: 
- GET
- POST
- PUT 
- DELETE
- UPDATE.

Dans notre cas nous allons utiliser la plupart du temps des GET et potentiellement des POST. 
- Le GET permet comme sont nom l'indique de récupérer des informations en fonction de certain paramètres. 
- Le POST nécéssite un envoie de données pour récupérer des données. Le body du post est, la plupart du temps, envoyé sous la forme d'un objet JSON.

Ces requêtes encapsulent un certain nombre de paramètres qui permettent soient d'identifier une provenance et un utilisateur ou de réaliser différentes actions.

In [2]:
import requests

In [3]:
url = "http://www.esiee.fr/"
response = requests.get(url)
response.status_code

200

Il existe deux méthodes pour récupérer le contenu de la page :

- `response.text` qui permet de retourner le texte sous la forme d'une chaine de charactères.
- `response.content` qui permet de récupérer le contenu de la page sous la forme de bytes

In [4]:
type(response.content)

bytes

In [5]:
type(response.text)

str

Pour récupérer les 1000 premiers charactères de la page :

In [6]:
response.text[0:1000]

'<!DOCTYPE html>\n<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->\n<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]-->\n<!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]-->\n<!--[if IE 9]>         <html class="no-js ie9"> <![endif]-->\n<!--[if gt IE 9]><!--> <html class="no-js"> <!--<![endif]-->\n<head profile="http://www.w3.org/1999/xhtml/vocab">\n\t<!-- Google Tag Manager -->\n\t<script>\n\t\t(function(w,d,s,l,i){\n\t\t\tw[l]=w[l]||[];w[l].push({\'gtm.start\':new Date().getTime(),event:\'gtm.js\'});\n\t\t\tvar f=d.getElementsByTagName(s)[0],\tj=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';\n\t\t\tj.async=true;\n\t\t\tj.src=\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n\t\t\t}\n\t\t)\n\t\t(window,document,\'script\',\'dataLayer\',\'GTM-MQF5LJB\');\n\t</script>\n\t<!-- End Google Tag Manager -->\n  <meta name="google-site-verification" content="JnG7DTdhQuWTeSHlWC63CeWpb3WValiOor

Pour récupérer les headers HTTP de la réponse :

In [7]:
response.headers["User-Agent"]

KeyError: 'user-agent'

On peut modifier les paramêtres de la requête et/ou ses headers. On peut par exemple ajouter un UserAgent et un timeout de 10 secondes:

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
response = requests.get(url, headers=headers, timeout = 10)
response.content[0:1000]

## Exercice

## Exercice 1

- Créer une classe Python permettant de faire des requêtes HTTP.
- Cette classe doit utiliser toujours le même UserAgent.
- Le TimeOut sera spécifié à chaque appelle avec une valeur par défaut.
- Un mécanisme de retry sera mis en place de façon recursive.

## Exercice 2

- Faire une fonction permettant de supprimer tous les espaces supperflus d'une string (???)
- Faire une fonction qui prend une string html et renvois une string intelligible (enlever les caractères spéciaux,)(???)
- Récupérer le domaine en fonction d'un url

In [8]:
#### Exercice 1
class requestClass : 
    def __init__(self):
        self.user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0"
    
    def request(self,url,timeout=10):
        import requests
        headers = {'User-Agent': self.user_agent}
        response = requests.get(url, headers=headers, timeout = timeout)
        while response.status_code != 200 : 
            response = requests.get(url, headers=headers, timeout = timeout)
        return response
        

In [9]:
test = requestClass()
test.request(url)

<Response [200]>

In [10]:
## Exercice 2
def getDomain(url):
    index_dash = [index  for index,char in enumerate(url) if char == "/"]
    if len(index_dash) > 2 : 
        # On retrouve un autre slash que ceux dans http:// 
        # le nom de domaine est donc compris entre le deuxième et troisième "/"
        return url[index_dash[1]+1:index_dash[2]]
    else : 
        # Il n'y a que les deux "/" de http://
        # Le nom de domaine est donc tous les caractères suivant le second "/"
        return url[index_dash[1]+1:]

getDomain(url)

'www.esiee.fr'

# Exploitation du HTML  

Ici, il faut récupérer le code HTML d'un site web à partir d'une requête. Lorsque vous avez récupéré le texte d'un site il faut le parser. Pour cela, on utilise BeautifulSoup qui permet de transformer la structure HTML en objet Python. Cela permet de récupérer efficacement les données qui nous intéresse.

Pour les webmasters, le blocage le plus souvent mis en place et un blocage sur le User-Agent. Le User-Agent est un paramètre intégré dans la requête HTTP réalisé par le Navigateur pour envoyer au front des informations basiques :

- la version du Navigateur,
- la version de l'OS
- Le type de gestionnaire graphique (Gecko)
- le type de device utilisé

Exemple de User Agent :  

`Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0`

Commençons à utiliser `BeautifulSoup`, pour l'installer : 

In [11]:
!pip install bs4
!pip install  lxml

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1271 sha256=7091a0bc6a4ba748cc0cd36a80c6ef927760c25f3bfcaba736c8455176f47b4e
  Stored in directory: /root/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [12]:
import requests
from bs4 import BeautifulSoup

Pour transformer une requête (requests) en objet BeautifulSoup :

In [119]:
response = requests.get(url)
soup = BeautifulSoup(response.text)

In [121]:
response.text

'<?xml version="1.0" encoding="UTF-8"?>\n<rss version="2.0" xmlns:atom="http://www.w3.org/2005/Atom" xmlns:media="http://search.yahoo.com/mrss/"><channel><title>Sport : Toute l’actualité sur Le Monde.fr.</title><description>Sport - Découvrez gratuitement tous les articles, les vidéos et les infographies de la rubrique Sport sur Le Monde.fr.</description><copyright>Le Monde - L’utilisation des flux RSS du Monde.fr est réservée à un usage strictement personnel, non professionnel et non collectif. Toute autre exploitation doit faire l’objet d’une autorisation et donner lieu au versement d’une rémunération. Contact : droitsdauteur@lemonde.fr</copyright><link>https://www.lemonde.fr/sport/rss_full.xml</link><pubDate>Tue, 23 Nov 2021 12:59:33 +0000</pubDate><language>fr</language><atom:link href="https://www.lemonde.fr/sport/rss_full.xml" rel="self" type="application/rss+xml"/><item><title><![CDATA[JO 2024\xa0: la justice autorise la reprise des travaux de construction d’une piscine d’entraîn

Il se peut qu'un message d'erreur arrive à ce point là si vous n'avez pas la librarie `lxml` installée, pour se faire vous avez juste à lancer la commande suivante : 

In [14]:
!pip install lxml

Pour trouver tous les liens d'une page on récupère la balise `a` qui permet de gérer les liens en HTML  :

In [15]:
soup.find_all("a")[0:10]

[<a href="#">
 <i class="icon-parametres"></i>
 </a>,
 <a href="https://gmail.com" target="_blank" title="Webmail ESIEE Paris"><i><img alt="" src="https://www.esiee.fr/sites/default/files/menu_icons/menu_icon_950.png"/> </i><span>Webmail ESIEE Paris</span></a>,
 <a href="https://planif.esiee.fr/direct/" target="_blank" title="Emploi du temps général"><i><img alt="" src="https://www.esiee.fr/sites/default/files/menu_icons/menu_icon_1331.png"/> </i><span>Emploi du temps général</span></a>,
 <a href="https://planif.esiee.fr/jsp/custom/esiee/easyMyPlanning.jsp" target="_blank" title="Emploi du temps individuel"><i><img alt="" src="https://www.esiee.fr/sites/default/files/menu_icons/menu_icon_1332.png"/> </i><span>Emploi du temps individuel</span></a>,
 <a href="https://intra.esiee.fr" target="_blank" title="Extranet"><i><img alt="" src="https://www.esiee.fr/sites/default/files/menu_icons/menu_icon_951.png"/> </i><span>Extranet</span></a>,
 <a href="https://blackboard.esiee.fr" target="_bla

On peut préciser la classe HTML voulue  pour l'ensemble des `a`:

```python
soup.find_all(class_="<CLASS_NAME>")[0:10]
```

Ici par exemple: 

In [16]:
soup.find_all(class_="slide")[0:5]

[<div class="slide slide-content">
 <span class="slide-content-date inline-block"><span class="date-display-single">18.11.2021</span></span>
 <span class="slide-content-theme inline-block is-uppercase">Concours</span>
 <div class="clearfix"></div>
 <div class="slide-content-img pull-left"><!-- scald=3191:news_thumbnail --><img alt="Etudiants ingénieurs ESIEE Paris lauréats HandiTech Trophy 2021" height="90" src="https://www.esiee.fr/sites/default/files/styles/news/public/thumbnails/image/visuel-handitech-trophy-920x600.jpg?itok=mXbmIdI6" title="Etudiants ingénieurs ESIEE Paris lauréats HandiTech Trophy 2021" width="120"/><!-- END scald=3191 --></div>
 <span class="slide-content-title"><a class="is-uppercase" href="/fr/actualite/handitech-trophy-echosign-2021">Des étudiant.es ESIEE Paris remportent le prix de la catégorie Étudiant du Handitech Trophy 2021</a></span>
 <div class="clearfix"></div>
 <p class="slide-content-desc">Le projet EchoSign a retenu l'attention du jury. Ce prix leur

Pour récupérer le text sans les balises HTML :

In [17]:
soup.text[0:1000]

' \n\n\n\n\n\n\n\n\n\nESIEE Paris, l’école de l’innovation technologique | Grande école d’ingénieurs | ESIEE Paris\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\nLes outils ESIEE Paris Webmail ESIEE Paris Emploi du temps général Emploi du temps individuel Extranet iCampus Blackboard Microsoft DreamSpark \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n menu\nFormations\n\n\nRETOUR menu\nFormations ESIEE ParisIngénieur\n\n\nIngénieur\nRETOUR Formations\nIngénieur ESIEE ParisPremier cycleLe cycle ingénieur\n\n\nLe cycle ingénieur\nRETOUR Ingénieur\nLe cycle ingénieur ESIEE ParisEnseignements de 1ère annéeEnseignements de 2e et 3e annéeProfils métiersFilières\n\n\nFilières\nRETOUR Ingénieur\n9 filières, 1 diplômeInformatiqueCybersécuritéDatascience et intelligence artificielleArtificial Intelligence and CybersecuritySystèmes embarquésSystèmes électroniques intelligentsGénie industrielBiotechnologies et e-santéEnergieDoubles-diplômesEmp

## Exercice
### Exercice 3

Améliorer la classe développé précédemment.

- Ajouter une méthode pour récupérer l'objet soup d'un url
- Récupérer une liste de User Agent et effectuer une rotation aléatoire sur celui à utiliser
- Utiliser cette classe pour parser une page HTML et récupérer : le titre, tous les H1 (si ils existent), les liens vers les images, les liens sortants vers d'autres sites, et le texte principal.

Parsing d'un sitemaps pour récupérer une listes de liens avec les informations disponibles. -> Stocker dans un dictionnaire et dans un fichier JSON local.

In [60]:
#### Exercice 3
class requestClass : 
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0"
    def __init__(self):
        self.response= None
        self.soup = None
    
    def request(self,url,timeout=10):
        import requests
        headers = {'User-Agent': requestClass.user_agent}
        response = requests.get(url, headers=headers, timeout = timeout)
        while response.status_code != 200 : 
            response = requests.get(url, headers=headers, timeout = timeout)
            
        self.response = response
        return self.response
    
    def get_soup(self,url=None):
        if self.response != None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.soup = BeautifulSoup(self.response.text)
            return self.soup
        # Sinon on doit faire la requête associée
        self.request(url)
        self.soup = BeautifulSoup(self.response.text)
        return self.soup
    
    def get_title(self,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        return self.soup.title.string
    
    def get_h1(self,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        return self.soup.find_all("h1")
   
    def get_img(self,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        images = self.soup.find_all("img")
        img_links = []
        for image in images:
            img_links.append(image.get("src"))
        return img_links
    
    def get_links(self,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        a_list = self.soup.find_all("a")
        links = []
        for a in a_list:
            links.append(a.get("href"))
        return links
    
    def get_p(self,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        p_list = self.soup.find_all("p")
        p_content = []
        for p in p_list:
            p_content.append(p.string)
        return p_content
    
    def get_tag(self,tag,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        tag_list = self.soup.find_all(tag)
        tag_content = []
        for tag in tag_list:
            tag_content.append(tag)
        return tag_content

In [61]:
test = requestClass()

In [62]:
print(test.get_title(url))
#print(test.get_h1(url))
#print(test.get_img(url))
#print(test.get_links(url))
#print(test.get_p(url))

Espace : Toute l’actualité sur Le Monde.fr.


In [63]:
print(test.get_tag("title",url))

[<title>Espace : Toute l’actualité sur Le Monde.fr.</title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>, <title></title>]


# Exploitation des appels d'API



Losque le front du site récupère des données sur une API géré par le back, un appel d'API est réalisé. Cet appel est recensé dans les appels réseaux. Il est alors possible de re-jouer cet appel pour récupérer à nouveau les données. Il est très facile de récupérer ces appels dans l'onglet Network de la console développeur de Chrome ou FireFox. La console vous permet de copier le code CURL pour effectuée et vous pouvez ensuite la transformer en code Python depuis le site https://curl.trillworks.com/.

Souvent les APIs sont bloquées avec certain paramètres. L'API verifie que dans les headers de la requêtes HTTP ces paramètres sont présents : * un token généré à la volée avec des protocole OAuth2 (ou moins développés). * un referer provenant du site web (la source de la requête), très facile à falsifier.



## Exercice 
### Exercice 4

- Utiliser les informations développées plus haut pour récupérer les premiers résultats d'une recherche d'une requête
sur Qwant. 

Tips : 

- Aller sur https://www.qwant.com/
- Ouvrir les outils de développements de Chrome ou Firefox
- Onglet Network
- Fouiller dans les requêtes

In [24]:
import requests,json

params = (
    ('q', 'esiee'),
    ('locale','fr_FR')
)
headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0"}

response = requests.get('https://api.qwant.com/v3/search/web', headers=headers, params=params)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://api.qwant.com/v3/search/web?q=esiee')

responseJSON = json.loads(response.text) 

In [43]:
responses = responseJSON["data"]["result"]["items"]["mainline"]
for response in responses: 
    if response["type"] == "web":
        sites = response["items"]
        for site in sites : 
            print(site["title"])

ESIEE Paris, l’école de l’innovation technologique | Grande école d’ingénieurs | ESIEE Paris
ESIEE
ESIEE-Amiens – École d'Ingénieurs à Amiens
ESIEE - Jour des Projets 2021
ESIEE Games
ESIEE-Amiens 2020/2021 - HYPERPLANNING
ESIEE : Avis, Formations et Informations !
esiee.paris: École d’ingénieurs ESIEE Paris, école de l’innovation | ESIEE Paris
esiee.fr: ESIEE Paris, l'école de l'innovation technologique
ESIEE IT - Découvrez les formations et le calendrier des stages de cet établissement - Ecole d'Ingénieurs


# Exercice Final  

Exercice Final
Utilisez tout ce que vous avez appris pour récupérer des articles de News avec une catégorie. Il est souvent intéressant de partir des flux RSS pour commencer :

Les données doivent comprendre :
- Le texte important propre
- L'url
- Le domaine
- la catégorie
- Le titre de l'article
- Le titre de la page
- (Facultatif) : les images

Tips : 

- Taper le nom de votre média favoris + RSS (par exemple : https://www.lemonde.fr/rss/)
- Aller dans le DOM de la page 
- Trouver les catégories et les liens vers les articles

In [159]:
class requestRSS : 
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:94.0) Gecko/20100101 Firefox/94.0"
    def __init__(self):
        self.response= None
        self.soup = None
        self.type="xml"
        self.title=None
    
    def request(self,url,timeout=10):
        import requests
        headers = {'User-Agent': requestClass.user_agent}
        response = requests.get(url, headers=headers, timeout = timeout)
        while response.status_code != 200 : 
            response = requests.get(url, headers=headers, timeout = timeout)
            
        self.response = response
        return self.response
    
    def get_soup(self,url=None):
        if self.response != None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.soup = BeautifulSoup(self.response.text,self.type)
            return self.soup
        # Sinon on doit faire la requête associée
        self.request(url)
        self.soup = BeautifulSoup(self.response.text,self.type)
        self.title=self.soup.title.string
        return self.soup
    
    def get_tag(self,tag,url=None):
        if self.soup == None :
            # Si on a déjà télécharger la page 
            # on se sert de la sauvegarde pour  éviter de réaliser une seconde requête
            self.get_soup(url)
        tag_list = self.soup.find_all(tag)
        tag_content = []
        for tag in tag_list:
            tag_content.append(tag)
        return tag_content
    
    def print_articles(self,url=None):
        articles = RSS.get_tag("item",url)
        for article in articles :
            print("==============================")
            print(self.title + "\n")
            print(article.title.string+"\n")
            print(article.description.string+"\n")
            print(article.find("media:content").get("url")+"\n")
            print(article.link.string)
            print("==============================")
        

In [160]:
RSS = requestRSS()
categorie = "sport"
url = "https://www.lemonde.fr/"+categorie+"/rss_full.xml"

In [161]:
RSS.print_articles(url)

Sport : Toute l’actualité sur Le Monde.fr.

JO 2024 : la justice autorise la reprise des travaux de construction d’une piscine d’entraînement à Aubervilliers

La cour administrative d’appel de Paris avait suspendu le permis de construire à la fin de septembre, invoquant des « doutes sérieux sur [sa] légalité », après une procédure d’urgence lancée par des opposants au projet.

https://img.lemde.fr/2021/11/21/560/0/6720/3360/644/322/60/0/077c489_151229577-211107jbalague-corps-isabelle-12.jpg

https://www.lemonde.fr/sport/article/2021/11/23/jo-2024-la-justice-autorise-la-reprise-des-travaux-de-construction-d-une-piscine-d-entrainement-a-aubervilliers_6103289_3242.html
Sport : Toute l’actualité sur Le Monde.fr.

Interruption du match OL-OM : Gérald Darmanin veut travailler sur les interdictions de stade

Les instances du football français se sont réunies au ministère de l’intérieur, mardi, pour commencer à tirer les conclusions de l’incident de dimanche, au Parc OL.

https://img.lemde.fr/

In [150]:
for article in articles :
    print("==============================")
    print(categorie.upper()+ " : "+article.title.string+"\n")
    print(article.description.string+"\n")
    print(article.find("media:content").get("url")+"\n")
    print(article.link.string)
print("==============================")

SPORT : Interruption du match OL-OM : Gérald Darmanin veut travailler sur les interdictions de stade

Les instances du football français se sont réunies au ministère de l’intérieur, mardi, pour commencer à tirer les conclusions de l’incident de dimanche, au Parc OL.

https://img.lemde.fr/2021/11/23/444/0/5329/2664/644/322/60/0/8f92317_5068366-01-06.jpg

https://www.lemonde.fr/societe/article/2021/11/23/interruption-du-match-ol-om-gerald-darmanin-veut-travailler-sur-les-interdictions-de-stade_6103285_3224.html
SPORT : Ski : entre affluence et protection de l’environnement, la station de Flaine sur une ligne de crête

La station haut-savoyarde entend faciliter l’accessibilité des skieurs grâce à l’ascenseur valléen Funiflaine, qui devrait être opérationnel en 2025. Parallèlement, elle multiplie les initiatives pour limiter son impact environnemental et préserver la biodiversité.

https://img.lemde.fr/2021/11/22/349/0/4187/2093/644/322/60/0/689f976_389623595-000-1e13wx.jpg

https://www.le